In [2]:
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

[NbConvertApp] Converting notebook plot_Taylor_normalized.ipynb to python
[NbConvertApp] Writing 4239 bytes to plot_Taylor_normalized.py


In [ ]:
def plt_Taylor_norm(data_ref,data_pred,
                mask=True,
                title='Normalized Taylor Diagram',
                label=['Observation', 'Model'],
                loc='right',
                tickRMSangle = 135.0,
                verbose = True,
                plot = True):
    '''
    Reference: observation
    Predicted: model data
    
    careful: here used different order as in input in skill_metrics: sm.taylor_statistics(pred,ref)
    
    Use this function to get Taylor statistics and plot them as normalized to std(reference) = 1
    
    Root mean sqare error are centered, see sm.centered_rms_dev? for details
    
    Applied normalization: 
        std_normalized(ref) = std(ref) / std(ref)
        std_normalized(pred) = std(pred) / std(ref)
        ... similar for CRMSD
        
    Input:
    data_ref: array with references/observation data
    data_pred: array with model data
    mask: bolean if observation data should be masked with np.where(data_pred !=0)
    title: string for plot title
    label: list with 2 entries for label for 'Observation', 'Model'
    loc: string, positional argument for legend
    tickRMSangle: used for ticks of RMSE in Taylor plot
    verbose: print calculated Taylor statistics
    plot: plot Taylor diagram
    
    Output:
    fig, sdev, crmsd, ccoef
    
    Changed style of marker in skill_metrics/plot_pattern_diagram_markers.py
    '''
    import numpy as np
    import matplotlib.pyplot as plt
    import skill_metrics as sm
                    
    if mask == True:
        # get statistics only from ocean gridpoints (where model data != 0)
        ind_stat = np.where(data_pred != 0)
        taylor_stats1 = sm.taylor_statistics(data_pred[ind_stat],data_ref[ind_stat])
    else:
        taylor_stats1 = sm.taylor_statistics(data_pred,data_ref)


    # original
    sdev = np.array([taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0],
                      taylor_stats1['crmsd'][1]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nOriginal Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))
    
    # normalized    
    sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                     taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])
    
    crmsd = np.array([taylor_stats1['crmsd'][0]/taylor_stats1['sdev'][0],
                      taylor_stats1['crmsd'][1]/taylor_stats1['sdev'][0]])
    
    ccoef = np.array([taylor_stats1['ccoef'][0], 
                      taylor_stats1['ccoef'][1]])

    if verbose:
        print('\nNormalized Taylor stats:\nSDEV pred: {0:6.5f}, ref: {1:6.5f}\nCRMS pred: {2:6.5f}, ref: {3:6.5f}\nCORRCOEF: {4:6.5f}'.format(
            sdev[1],sdev[0],crmsd[1],crmsd[0],ccoef[1], ccoef[0]))

    if plot:
        fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
        sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                              titleOBS = 'observation', markerLabel = label,
                              markerLabelColor = 'c',
                              markerColor = 'c', markerLegend = 'on',
                              tickRMS = [0,0.25,0.5,0.75,1.25],
                              tickRMSangle = tickRMSangle,
                              colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                              titleRMS = 'off', tickSTD = [0,0.25,0.5,0.75,1.25],
                              axismax = 1.25, 
                              colSTD = 'b', styleSTD = '-.',
                              widthSTD = 1.0, titleSTD = 'on',
                              colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                              titleCOR = 'on')

        plt.title(title, loc=loc)   
    
    else:
        fig = 'none'

    return fig, sdev, crmsd, ccoef